# Base de nascidos vivos do dataSUS

In [1]:
import pandas as pd
import requests
import numpy as np

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

FileNotFoundError: [Errno 2] No such file or directory: 'SINASC_RO_2019.csv'

In [ ]:
# 2) criando um DataFrame e contando valores missing em geral
df = pd.DataFrame(sinasc)
df.isna().sum()


In [ ]:
# 9) Renomeando as colunas para snakecase
df.rename(columns={
    'LOCNASC': 'loc_nasc',
    'IDADEMAE': 'idade_mae',
    'ESTCIVMAE': 'est_civ_mae',
    'ESCMAE': 'esc_mae',
    'QTDFILVIVO': 'qtd_fil_vivo',
    'GESTACAO': 'gestacao',
    'GRAVIDEZ': 'gravidez',
    'CONSULTAS': 'consultas',
    'APGAR5': 'apgar5'
}, inplace=True)

In [2]:
# 3) separando e contando os valores missing das colunas de interesse
colunas_interessantes = ['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo', 
    'gestacao', 'gravidez', 'consultas', 'apgar5'] 
colunas_interessantes = df[colunas_interessantes]
colunas_interessantes.isna().sum()

NameError: name 'df' is not defined

In [398]:
# 4) apagando os valores missing da coluna 'APGAR5'
df.dropna(subset=['apgar5'], inplace=True)
colunas_interessantes = df[colunas_interessantes.columns]
colunas_interessantes.isna().sum()


loc_nasc           0
idade_mae          0
est_civ_mae      315
esc_mae          310
qtd_fil_vivo    1566
gestacao        1216
gravidez          76
consultas          0
apgar5             0
dtype: int64

In [399]:
# 5) substituindo os valores missing por 9.0
df['est_civ_mae'].fillna(9.0, inplace=True)
df['consultas'].fillna(9.0, inplace=True)
colunas_interessantes = df[colunas_interessantes.columns]
colunas_interessantes.isna().sum()

loc_nasc           0
idade_mae          0
est_civ_mae        0
esc_mae          310
qtd_fil_vivo    1566
gestacao        1216
gravidez          76
consultas          0
apgar5             0
dtype: int64

In [400]:
# 6) Substituindo os valores missing da coluna 'QTDFILVIVO', por zero e conferindo a quantidade de valores missing
df['qtd_fil_vivo'].fillna(0, inplace=True)
colunas_interessantes = df[colunas_interessantes.columns]
colunas_interessantes.isna().sum()
print('Os valores faltantes foram substituídos por zero e a quantidade de valores faltantes é:', df['qtd_fil_vivo'].isna().sum())

Os valores faltantes foram substituídos por zero e a quantidade de valores faltantes é: 0


In [401]:
# 7) Verificando mais colunas que precisam ser alteradas e substituindo valores missing, escolaridade missing = 1 = Nenhuma, Gravidez = 1 = Única o restante está como ignorado.
df['loc_nasc'].fillna(9.0, inplace=True)
df['esc_mae'].fillna(1, inplace=True) 
df['gestacao'].fillna(9.0, inplace=True)
df['gravidez'].fillna(1, inplace=True)
colunas_interessantes = df[colunas_interessantes.columns]
colunas_interessantes.isna().sum()

loc_nasc        0
idade_mae       0
est_civ_mae     0
esc_mae         0
qtd_fil_vivo    0
gestacao        0
gravidez        0
consultas       0
apgar5          0
dtype: int64

In [408]:
# 8) classificando como 'asfixia normal', 'asfixia leve', 'asfixia moderada' e 'asfixia severa' de acordo com a nota do APGAR5
df.loc[df['apgar5'] >= 8, 'apgar5_cat'] = 'asfixia normal'
df.loc[(df['apgar5'] >= 6) & (df['apgar5'] <= 7), 'apgar5_cat'] = 'asfixia leve'
df.loc[(df['apgar5'] >= 4) & (df['apgar5'] <= 5), 'apgar5_cat'] = 'asfixia moderada'
df.loc[df['apgar5'] <= 3, 'apgar5_cat'] = 'asfixia severa'

freq_apgar5_cat = df['apgar5_cat'].value_counts()
print('De acordo com as notas do APGAR5, essas foram as quantidadesde cada uma das asfixias:\n')
print(freq_apgar5_cat)

De acordo com as notas do APGAR5, essas foram as quantidadesde cada uma das asfixias:

asfixia normal      26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: apgar5_cat, dtype: int64


In [409]:
colunas_interessantes

,loc_nasc,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0
